# Testing the models:

In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from matplotlib import pyplot as plt

In [3]:
X = np.load("../train_test_data/train_x_values.npy")
y = np.load("../train_test_data/train_y_values.npy")

## Encoding the target

In [4]:
ohe = OneHotEncoder(sparse=False)
ohe.fit(y.reshape(-1,1))

OneHotEncoder(sparse=False)

In [5]:
y_encoded = ohe.transform(y.reshape(-1,1))# Display the detected categories
print(f"The categories detected by the OneHotEncoder are {ohe.categories_}")

The categories detected by the OneHotEncoder are [array(['gametocyte', 'leukocyte', 'red blood cell', 'ring', 'schizont',
       'trophozoite'], dtype='<U14')]


In [6]:
y_encoded_df=pd.DataFrame(y_encoded, columns = ohe.get_feature_names_out())

In [7]:
y_encoded_df

,x0_gametocyte,x0_leukocyte,x0_red blood cell,x0_ring,x0_schizont,x0_trophozoite
0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...
995,0.0,0.0,1.0,0.0,0.0,0.0
996,0.0,0.0,1.0,0.0,0.0,0.0
997,0.0,0.0,1.0,0.0,0.0,0.0
998,0.0,0.0,1.0,0.0,0.0,0.0


## Enconding the test target

In [8]:
X_test = np.load("../train_test_data/test_x_values.npy")
y_test = np.load("../train_test_data/test_y_values.npy")

In [9]:
y_test_encoded = ohe.transform(y_test.reshape(-1,1))

## Balancing the data using class weights

In [10]:
from sklearn.utils import class_weight

In [11]:
cw = y_encoded_df.sum().to_dict()
cw

{'x0_gametocyte': 3.0,
 'x0_leukocyte': 2.0,
 'x0_red blood cell': 972.0,
 'x0_ring': 5.0,
 'x0_schizont': 2.0,
 'x0_trophozoite': 16.0}

In [12]:
(np.unique(y_encoded, axis = 0))

array([[0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.]])

In [13]:
cwv = class_weight.compute_class_weight('balanced', 
                            classes = np.unique(y),
                            y=y)

In [14]:
z = zip([int(i) for i in (np.arange(0,7))], 
            cwv)
np.unique(y_encoded)
[int(i) for i in (np.arange(0,7))]
weighting = dict(z)
weighting

{0: 55.55555555555556,
 1: 83.33333333333333,
 2: 0.17146776406035666,
 3: 33.333333333333336,
 4: 83.33333333333333,
 5: 10.416666666666666}

In [15]:
X_expanded = np.expand_dims(X,axis=3)
image_shape = (X_expanded.shape[1], X_expanded.shape[2], X_expanded.shape[3])

In [16]:
X_expanded=X_expanded/255 #.shape

## MODEL3 - with L1 regularization:

In [17]:
# ARCHITECTURE imports
import numpy as np
from tensorflow.keras import  Sequential, layers, regularizers

In [18]:
reg_l1 = regularizers.L1(0.01)

In [ ]:
#### 1. ARCHITECTURE

model3 = Sequential()

model3.add(layers.Conv2D(filters=128, kernel_size=(3,3), input_shape=image_shape, activation='relu'))
model3.add(layers.MaxPooling2D(pool_size=(2, 2)))

model3.add(layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu', kernel_regularizer=reg_l1))
model3.add(layers.MaxPooling2D(pool_size=(2, 2)))

model3.add(layers.Conv2D(filters=32, kernel_size=(2,2), activation='relu', kernel_regularizer=reg_l1))
model3.add(layers.MaxPooling2D(pool_size=(2, 2)))

model3.add(layers.Flatten())

model3.add(layers.Dense(128, activation='relu', kernel_regularizer=reg_l1))

# Dropouts reduce overfitting by randomly turning neurons off during training.

model3.add(layers.Dropout(0.2))

# categorical = softmax
model3.add(layers.Dense(6, activation='softmax'))

In [ ]:
# COMPILATION imports

from tensorflow.keras import optimizers, callbacks
from tensorflow.keras.metrics import Recall

In [ ]:
#### 2. COMPILATION

adam = optimizers.Adam(learning_rate=0.01, beta_1=0.9, beta_2=0.999)

model3.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=['accuracy', Recall()])

In [ ]:
from tensorflow.keras import callbacks

In [ ]:
#### 3. FIT

es = callbacks.EarlyStopping(patience=30, restore_best_weights=True)

history3 = model3.fit(X_expanded, y_encoded,
          batch_size=32, # Batch size -too small--> no generalization
          epochs=500,    #            -too large--> slow computations
          validation_split=0.2,
          callbacks=[es],
          class_weight = weighting,
          verbose=0)

In [ ]:
def plot_loss(history3, label, n):
   plt.semilogy(history3.epoch, history3.history['loss'],
                color='b', label='Train ' + label)
   plt.semilogy(history3.epoch, history3.history['val_loss'],
                color='r', label='Val ' + label,
                linestyle="--")
   plt.xlabel('Epoch')
   plt.ylabel('Loss')

In [ ]:
plot_loss(history3,"first run",'b')